## Target number of units and capacity

In [ ]:
import os 
import os.path
import sys
cwd = os.getcwd()
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir)

In [ ]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import utils
import orca
# import datasources
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
scenarios = utils.yaml_to_dict('../data/scenario_config.yaml', 'scenario')
print(scenarios)

##### from urbansim.parcel

In [ ]:
parcel_capacity_sql = '''
SELECT [parcel_id]
      ,[jurisdiction_id]  
      ,[cap_jurisdiction_id]
      ,[du_2015]
      ,[du_2017]
      ,[capacity_1]
      ,[capacity_2]
      ,[site_id]
  FROM [urbansim].[urbansim].[parcel]
  WHERE site_id IS NULL'''
capacity_df = pd.read_sql(parcel_capacity_sql,mssql_engine)
urbansim_parcel_capacity = int(capacity_df.capacity_2.sum())
print("\nCapacity from urbansim.parcel: {:,}".format(urbansim_parcel_capacity))

##### from urbansim.additional_capacity

In [ ]:
assigned_parcel_sql = '''
SELECT  a.parcel_id,
        p.mgra_id, 
        jur_id as cap_jurisdiction_id,
        jur_id as jurisdiction_id,
        p.luz_id,
        p.site_id,
        a.du,
        p.du_2017 as residential_units,
        0 as partial_build,
        type
  FROM [urbansim].[urbansim].[additional_capacity] a
  join urbansim.parcel p on p.parcel_id = a.parcel_id
  where version_id = %s
'''
assigned_parcel_sql = assigned_parcel_sql % scenarios['additional_capacity_version']
assigned_df = pd.read_sql(assigned_parcel_sql, mssql_engine)
assigned_capacity = int(assigned_df.du.sum())
print("\nCapacity from urbansim.additional_capacity: {:,}".format(int(assigned_df.du.sum())))

##### from urbansim.scheduled_development_parcel 

In [ ]:
sched_dev_parcel_sql = '''
SELECT [capacity_3]
      ,[sfu_effective_adj]
      ,[mfu_effective_adj]
      ,[mhu_effective_adj]
      ,[notes]
      ,[editor]
  FROM [urbansim].[urbansim].[scheduled_development_parcel]
'''
sched_dev_df = pd.read_sql(sched_dev_parcel_sql, mssql_engine)
site_from_sched_dev_parcel = int(sched_dev_df.capacity_3.sum())
print("\n   Scheduled development capacity from urbansim.scheduled_development_parcel : {:,}".format(int(site_from_sched_dev_parcel)))

##### urbansim.urbansim.scheduled_development_do_not_use

In [ ]:
sched_dev_do_not_use_sql = '''
SELECT [sched_version_id]
      ,[parcel_id]
      ,[yr]
      ,[site_id]
      ,[capacity_3]
  FROM [urbansim].[urbansim].[scheduled_development_do_not_use]'''
site_do_not_use = pd.read_sql(sched_dev_do_not_use_sql,mssql_engine)
site_from_do_not_use = int(site_do_not_use.capacity_3.sum())
print("\n  Scheduled developments from scheduled_development_do_not_use: {:,}".format(site_from_do_not_use))

#### Total Capacity

In [ ]:
total_capacity = urbansim_parcel_capacity + assigned_capacity + site_from_sched_dev_parcel
print("\nTotal Capacity: {:,}".format(total_capacity))
print("     = {:,} + {:,} + {:,}\n".format(urbansim_parcel_capacity,assigned_capacity,site_from_sched_dev_parcel))
print("     Capacity from urbansim.parcel and sched dev = {:,}\n".format(urbansim_parcel_capacity+site_from_sched_dev_parcel))

#### Target units to add 2050 

##### isam.economic_output.urbansim_housing_units

In [ ]:
units_needed_sql = '''
SELECT [yr]
      ,[version_id]
      ,[housing_units_add]
  FROM [isam].[economic_output].[urbansim_housing_units]
  WHERE version_id = %s'''
units_needed_sql = units_needed_sql % scenarios['demographic_simulation_id']
units_needed_df =  pd.read_sql(units_needed_sql, mssql_engine)
units_needed_df['total_housing_units_add'] = units_needed_df['housing_units_add'].cumsum()
units_needed = int(sum(units_needed_df.housing_units_add))
print("\n  Target units to add : {:,}\n\
      (from isam.economic_output.urbansim_housing_units demographic simulation id {:})".\
      format(units_needed,scenarios['demographic_simulation_id']))

In [ ]:
# compare to 364,374

#### Remaining Capacity at 2050

In [ ]:
print("\n Remaining capacity 2050: {:,}\n       = {:} capacity - {:} units needed\n".\
format(total_capacity - units_needed,total_capacity,units_needed))

## Capacity from scheduled developments

##### urbansim.scheduled_development_do_not_use

In [ ]:
print("\n  Scheduled developments from scheduled_development_do_not_use: {:,}".format(site_from_do_not_use))

In [ ]:
print("\n   Scheduled developments from urbansim.scheduled_development_parcel : {:,}".format(int(site_from_sched_dev_parcel)))

### check that no parcels with scheduled developments have additional capacity

##### with urbansim.scheduled_development_parcel

In [ ]:
sched_dev_in_urbansim_parcel_sql = '''
select parcel_id,site_id,capacity_2 from urbansim.parcel where site_id IS NULL and parcel_id IN (
SELECT parcel_id from urbansim.scheduled_development_parcel)'''
sched_dev_in_urbansim_parcel = pd.read_sql(sched_dev_in_urbansim_parcel_sql, mssql_engine)
print(sched_dev_in_urbansim_parcel)

##### with scheduled_development_do_not_use

In [ ]:
sched_dev_in_urbansim_parcel_sql = '''
select parcel_id,site_id,capacity_2 from urbansim.parcel where site_id IS NULL and parcel_id IN (
SELECT parcel_id from urbansim.scheduled_development_do_not_use)'''
sched_dev_in_urbansim_parcel = pd.read_sql(sched_dev_in_urbansim_parcel_sql, mssql_engine)
print(sched_dev_in_urbansim_parcel)

#### check that parcels do not have capacity 2 and capacity 3

In [ ]:
capacity_2_and_3_sql = '''select cap_jurisdiction_id, p.parcel_id,p.site_id,capacity_2,s.site_id,capacity_3 
from urbansim.parcel p
join urbansim.scheduled_development_do_not_use s
on s.parcel_id = p.parcel_id
where capacity_2 > 0 and p.site_id IS NULL and s.capacity_3 > 0
ORDER BY jurisdiction_id'''
capacity_2_and_3 = pd.read_sql(capacity_2_and_3_sql, mssql_engine)
print(capacity_2_and_3)

# Housing units 2017

In [ ]:
du_2017 = capacity_df.du_2017.sum()
print("\n Housing units 2017 from urbansim.parcel: {:,}\n".format(du_2017))

# Housing units 2050 (forecast)

In [ ]:
du_2017 = capacity_df.du_2017.sum()
print("\n Housing units 2050: {:,}\n\n (= {:,} du 2017 +  {:,} units needed) \n".\
      format(du_2017 + units_needed,du_2017,units_needed))  

#### Output of simulation compared to units needed

In [ ]:
# get max run id from urbansim
run_id_sql = '''
SELECT max(run_id)
  FROM [urbansim].[urbansim].[urbansim_lite_output]
'''
run_id_df = pd.read_sql(run_id_sql, mssql_engine)
run_id = int(run_id_df.values)
print("\n   Max run id : {:,}".format(run_id))

hs_change_sql = '''
    SELECT o.parcel_id, j.name,  p.cap_jurisdiction_id, p.jurisdiction_id, p.mgra_id, p.luz_id,
    unit_change as hs_change, source, capacity_type,year_simulation
      FROM urbansim.urbansim.urbansim_lite_output o 
      JOIN urbansim.urbansim.parcel p on p.parcel_id = o.parcel_id
      JOIN urbansim.ref.jurisdiction j on p.cap_jurisdiction_id = j.jurisdiction_id
     WHERE run_id =  %s
  ORDER BY j.name,p.jurisdiction_id, year_simulation'''
hs_change_sql = hs_change_sql % run_id
hs = pd.read_sql(hs_change_sql,mssql_engine)
print("\n   Units added: {:,}".format(int(hs.hs_change.sum())))

In [ ]:
units_added_by_yr = pd.DataFrame({'units_per_year': hs.
                                          groupby(["year_simulation"])
                                          .hs_change.sum()}).reset_index()

In [ ]:
pd.merge(units_needed_df[['yr','housing_units_add']],units_added_by_yr,left_on='yr',right_on='year_simulation')

# Output of simulation compared to capacities

In [ ]:
output_and_capacity_sql =  '''
 WITH u_parcel_jur AS (
 SELECT parcel_id, cap_jurisdiction_id,capacity_2, 'jur' as capacity_type
 FROM urbansim.parcel
 WHERE capacity_2 > 0 and site_id IS NULL),
 u_output AS (
  SELECT  [parcel_id],capacity_type,sum(unit_change) as units_added 
  FROM [urbansim].[urbansim].[urbansim_lite_output]
  where run_id = %s
  GROUP BY parcel_id, capacity_type),
  assigned AS (
  SELECT [parcel_id],jur_id
      ,[type] as capacity_type
      ,[du] as capacity_4
  FROM [urbansim].[urbansim].[additional_capacity]),
  sched AS (
  SELECT [parcel_id],
      sum([capacity_3]) as capacity_3,
      'sched_dev' as capacity_type
  FROM [urbansim].[urbansim].[scheduled_development_do_not_use]
  WHERE capacity_3 > 0
  GROUP BY parcel_id )
  SELECT o.parcel_id as o_parcel_id, units_added, 
        p.parcel_id as p_parcel_id, p.cap_jurisdiction_id,p.capacity_2 as urbansim_parcel_capacity,
        p.capacity_type as p_capacity_type,
        a.parcel_id as a_parcel_id,a.jur_id,a.capacity_4 as assigned_capacity,
        a.capacity_type as a_capacity_type,
        s.parcel_id as s_parcel_id, s.capacity_3 as sched_dev_capacity,
        s.capacity_type as s_capacity_type       
  FROM u_output o
  FULL OUTER JOIN  u_parcel_jur p
  ON o.parcel_id = p.parcel_id and o.capacity_type = p.capacity_type
  FULL OUTER JOIN assigned a
  ON a.parcel_id = o.parcel_id and a.capacity_type = o.capacity_type
    FULL OUTER JOIN sched s
  ON s.parcel_id = o.parcel_id and s.capacity_type = o.capacity_type
  --WHERE units_added IS NULL
  ORDER BY  p.capacity_2'''

In [ ]:
output_and_capacity_sql = output_and_capacity_sql % run_id
output_and_capacity = pd.read_sql(output_and_capacity_sql, mssql_engine)

In [ ]:
# output_and_capacity.to_csv('test.csv')

In [ ]:
total_units_added = int(output_and_capacity.units_added.sum())
print("\nTotal Units added: {:,}".format(total_units_added))
units_from_sched_dev =  int(output_and_capacity.loc[output_and_capacity.s_capacity_type=='sched_dev'].units_added.sum())
print("\n   Units added from sched dev: {:,}".format(units_from_sched_dev))
units_from_assigned =  int(output_and_capacity.loc[output_and_capacity.\
                                                   a_capacity_type.isin(['adu','mc','cc','uc','tc','tco'])].\
                           units_added.sum())
print("\n   Units added from assigned: {:,}".format(units_from_assigned))
units_from_urbansim_parcel =  int(output_and_capacity.loc[output_and_capacity.p_capacity_type=='jur'].units_added.sum())
print("\n   Units added from urb parcel: {:,}".format(units_from_urbansim_parcel))
sum_units = units_from_sched_dev + units_from_assigned + units_from_urbansim_parcel
print("\n   Sum : {:,}".format(sum_units))

In [ ]:
print("\nCapacity from scheduled_development_do_not_use: {:,}".format(site_from_do_not_use))
print("\nCapacity from urbansim.additional_capacity: {:,}".format(int(assigned_df.du.sum())))
print("\nCapacity from urbansim.parcel: {:,}".format(int(capacity_df.capacity_2.sum())))
sum_capacity = site_from_do_not_use + int(assigned_df.du.sum()) + int(capacity_df.capacity_2.sum())
print("\nSum : {:,}".format(sum_capacity))

In [ ]:
sched_dev_difference = site_from_do_not_use - units_from_sched_dev
assigned_difference = int(assigned_df.du.sum()) - units_from_assigned
urbansim_parcel_difference = int(capacity_df.capacity_2.sum()) - units_from_urbansim_parcel
total_difference = sum_capacity - sum_units
print("\nDifference from scheduled_development_do_not_use: {:,}".format(sched_dev_difference))
print("\nDifference from urbansim.additional_capacity: {:,}".format(assigned_difference))
print("\nDifference from urbansim.parcel: {:,}".format(urbansim_parcel_difference))
print("\nTotal Difference : {:,}".format(total_difference))


In [ ]:
urbansim_parcel_unused = output_and_capacity.loc[output_and_capacity.p_capacity_type == 'jur'].copy()
urbansim_parcel_unused.fillna(0,inplace=True)
urbansim_parcel_unused['unused'] = urbansim_parcel_unused.urbansim_parcel_capacity - urbansim_parcel_unused.units_added

In [ ]:
urbansim_parcel_unused['unused'].sum()

In [ ]:
assigned_unused = output_and_capacity.loc[output_and_capacity.a_capacity_type.\
                                                 isin(['adu','mc','cc','uc','tc','tco'])].copy()
assigned_unused.fillna(0,inplace=True)
assigned_unused['unused'] = assigned_unused.assigned_capacity - assigned_unused.units_added

In [ ]:
assigned_unused['unused'].sum()

In [ ]:
urbansim_parcel_unused.loc[urbansim_parcel_unused.unused!=0]\
[['p_parcel_id','cap_jurisdiction_id','units_added','urbansim_parcel_capacity']].sort_values(by='cap_jurisdiction_id')

In [ ]:
assigned_unused.loc[assigned_unused.unused!=0]\
[['a_parcel_id','jur_id','units_added','assigned_capacity']].sort_values(by='jur_id')

# Plot (note: increase 2026 and 2027)

In [ ]:
hu_df = units_needed_df[['yr','households','housing_units_add']].copy()

In [ ]:
hu_df['capacity'] = total_capacity

In [ ]:
hu_df['total_housing_units_add'] = hu_df['housing_units_add'].cumsum()

In [ ]:
hu_df.head()

In [ ]:
hu_df['remaining_capacity'] = hu_df['capacity'] - hu_df['total_housing_units_add'] 

In [ ]:
fig, axes = plt.subplots(3, 1)


plot_df = hu_df[['yr','housing_units_add']].copy()
plot_df.rename(columns = {'housing_units_add': 'Housing units to add'}, inplace=True)
plot_df.set_index('yr',inplace=True)
plot_df.plot(style='.-',ax=axes.flat[0],figsize=(10,5),color='b')

plot_df = hu_df[['yr','total_housing_units_add','households']].copy()
plot_df.set_index('yr',inplace=True)
plot_df['Total Housing Units'] = du_2017 + plot_df['total_housing_units_add']
plot_df.rename(columns = {'households': 'Households'}, inplace=True)
del  plot_df['total_housing_units_add']
plot_df.plot(style='.-',ax=axes.flat[1],figsize=(10,10),color=['darkorange','b'])

plot_df = hu_df[['yr','total_housing_units_add','remaining_capacity']].copy()
plot_df.set_index('yr',inplace=True)
plot_df.rename(columns = {'total_housing_units_add': 'Housing units'}, inplace=True)
plot_df.rename(columns = {'remaining_capacity': 'Remaining capacity'}, inplace=True)
plot_df.plot(style='.-',ax=axes.flat[2],figsize=(10,15),color=['b','chocolate'])
plt.axhline(y=0,color='black', lw=2, alpha=0.5,linestyle='--')
# format
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

axes.flat[0].set_xlabel("Simulation Year",size =16)
axes.flat[0].set_ylabel("Housing Units",size =16)
ptitle = 'Housing Units Needed by Year\n demographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[0].set_title(ptitle,size=16) 
axes.flat[0].yaxis.set_major_formatter(tick) 
axes.flat[0].legend(fontsize=14)

# change order of legend in second plot
axes.flat[1].set_xlabel("Simulation Year",size=16)
axes.flat[1].set_ylabel("Total Housing Units",size =16)
ptitle = 'Housing Units and Households \ndemographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[1].set_title(ptitle,size=16) 
axes.flat[1].yaxis.set_major_formatter(tick) 
handles, labels = axes.flat[1].get_legend_handles_labels()
zipped = zip(labels,handles)
zipped.sort(key = lambda t: t[0],reverse=True)
labels, handles = zip(*zipped)
axes.flat[1].legend(handles, labels,fontsize=14)
# axes.flat[1].legend(fontsize=14)

axes.flat[2].set_xlabel("Simulation Year",size =16)
axes.flat[2].set_ylabel("Housing Units (cusum)",size =16)
ptitle = 'Housing Units Needed and Remaining Capacity\n demographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[2].set_title(ptitle,size=16) 
axes.flat[2].yaxis.set_major_formatter(tick) 
axes.flat[2].legend(fontsize=14)

plt.xticks(rotation=25)
plt.tight_layout(pad=0.5, w_pad=0.6, h_pad=2)



plt.show()
fig.savefig('Demographic_simulation_2001.png', format='png', dpi=300)

# Map with folium

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[32.7157,  -117.1611])

In [ ]:
m